In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),
                    nn.ReLU(),
                    nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

tensor([[0.4144],
        [0.3488]], grad_fn=<AddmmBackward0>)

In [7]:
# net[2]拿到的是nn.Linear(8,1)，state_dict是权重和偏差
print(f'第二层网络的状态（包括权重和偏差）：{net[2].state_dict()}')
# bias是一个Parameter，也就是可优化的参数
print(f'偏差的类型：{type(net[2].bias)}')
print(f'查看偏差（包括值和梯度）：{net[2].bias}')
print(f'查看偏差的值（data）：{net[2].bias.data}')
print(f'查看偏差的梯度（grad）：{net[2].bias.grad}\n(由于没有做梯度下降所以grad=None)')

第二层网络的状态（包括权重和偏差）：OrderedDict([('weight', tensor([[-0.0158, -0.3498,  0.2600,  0.2497, -0.0741,  0.1854,  0.3060, -0.1925]])), ('bias', tensor([0.3488]))])
偏差的类型：<class 'torch.nn.parameter.Parameter'>
查看偏差（包括值和梯度）：Parameter containing:
tensor([0.3488], requires_grad=True)
查看偏差的值（data）：tensor([0.3488])
查看偏差的梯度（grad）：None
(由于没有做梯度下降所有grad=None)


In [10]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
print(net.state_dict())
net.state_dict()['2.bias'].data

OrderedDict([('0.weight', tensor([[-0.2472,  0.0148, -0.1239,  0.4094],
        [ 0.2604,  0.1003, -0.4057,  0.1262],
        [-0.1069,  0.1446,  0.4027,  0.2727],
        [-0.2044,  0.4485,  0.0592, -0.4451],
        [-0.0709,  0.0416, -0.0018, -0.3404],
        [-0.3940, -0.0842,  0.0229, -0.2976],
        [ 0.0410, -0.4246, -0.1798, -0.4027],
        [-0.4664, -0.2219, -0.1327, -0.1411]])), ('0.bias', tensor([-0.1172, -0.1561, -0.3075, -0.3918,  0.0839, -0.3010,  0.1176, -0.3494])), ('2.weight', tensor([[-0.0158, -0.3498,  0.2600,  0.2497, -0.0741,  0.1854,  0.3060, -0.1925]])), ('2.bias', tensor([0.3488]))])


tensor([0.3488])

In [13]:
def block1():
    return nn.Sequential(nn.Linear(4,8),
                         nn.ReLU(),
                         nn.Linear(8,4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 传入4个block1
        # f'block {i}':传一个字符串的名字
        net.add_module(f'block {i}',block1())
    return net

rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.1193],
        [0.1193]], grad_fn=<AddmmBackward0>)

In [14]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [19]:
def init_normal(m):
    '''
    用正态分布初始化权重，偏差初始化为0
    :param m: 层
    :return:
    '''
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)

# 对于net里面的所有module调用init_normal
# 如果是嵌套就是嵌套遍历
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0126, -0.0043, -0.0059, -0.0189]), tensor(0.))

In [20]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0664,  0.3901,  0.4517, -0.6281])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [28]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name,param.shape) for name,param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight,-10,10)
        # 将绝对值小于5的权重置零
        m.weight.data *= m.weight.data.abs()>=5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000, 0.0000, 0.0000],
        [-0.0000, -0.0000, 9.9754, 8.4712]], grad_fn=<SliceBackward0>)

In [29]:
# 直接取出替换
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

In [30]:
# 参数绑定
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                    shared,nn.ReLU(),
                    shared,nn.ReLU(),
                    nn.Linear(8,1))
net(X)
# 两个share是一样的
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
